<a href="https://colab.research.google.com/github/ccsed/ProgettoLabAI/blob/main/Progetto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collego colab a google drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importo le librerie necessarie

In [2]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 56.8 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import rasterio
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
import albumentations as A
from PIL import Image
import matplotlib.pyplot as plt
import random
import skimage.io as io
from rasterio.plot import show
import torch.nn as nn
import torchvision.transforms.functional as TF
from albumentations.pytorch import ToTensorV2
import torch.optim as optim
from tqdm import tqdm
import torchvision
from osgeo import gdal
import cv2
import tifffile as tiff
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import copy

Definisco alcune directory

In [4]:
drive_dir = '/content/drive/MyDrive'
image_dir = '/content/drive/MyDrive/Data/SN6_buildings_AOI_11_Rotterdam_train/train/AOI_11_Rotterdam/SAR-Intensity'
train_path = '/content/drive/MyDrive/train.txt'
val_path = '/content/drive/MyDrive/val.txt'
test_path = '/content/drive/MyDrive/test.txt'

Media e std calcolate in Funzioni.ipynb

In [5]:
# mean = [22.79312241, 24.30391245, 19.29892107, 17.69439361]
# std = [16.42519281, 17.42876255, 14.27711349, 13.24714869]
mean = [0.0, 0.0, 0.0, 0.0]
std = [1.0, 1.0, 1.0, 1.0]

Definiamo la classe SARDataset

In [6]:
# class SARDataset(Dataset):
#     def __init__(self, image_paths, mask_paths, transform=None):
#         self.image_paths = image_paths
#         self.mask_paths = mask_paths
#         self.transform = transform

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         image_path = self.image_paths[idx]
#         mask_path = self.mask_paths[idx]


#         with rasterio.open(image_path) as src:
#             image = src.read().transpose((1, 2, 0))
#             print(image.shape)
#             image = image.astype(np.float32)

#         with rasterio.open(mask_path) as src:
#             mask = src.read(1)
#             print(mask.shape)
#             mask[mask == 255.0] = 1.0


#         if self.transform is not None:
#             augmentations = self.transform(image=image, mask=mask)
#             image = augmentations["image"]
#             mask = augmentations["mask"]

#         return image, mask

class SARDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]


        image = tiff.imread(image_path)

        mask = tiff.imread(mask_path)
        if mask.ndim == 3:
            mask = mask[:, :, 0]
        mask = mask.astype(np.float32)
        mask[mask == 255.0] = 1.0


        if image.shape[:2] != mask.shape:
            raise ValueError(f"Le dimensioni dell'immagine {image_path} e della maschera {mask_path} non corrispondono.")

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

Carico i percorsi delle immagini per il training, il validation e il test set

In [7]:
with open(train_path, 'r') as f:
    train_data = [line.split() for line in f.read().splitlines()]

train_image_paths = [line[0] for line in train_data]
train_label_paths = [line[1] for line in train_data]

with open(val_path, 'r') as f:
    val_data = [line.split() for line in f.read().splitlines()]

val_image_paths = [line[0] for line in val_data]
val_label_paths = [line[1] for line in val_data]

with open(test_path, 'r') as f:
    test_data = [line.split() for line in f.read().splitlines()]

test_image_paths = [line[0] for line in test_data]
test_label_paths = [line[1] for line in test_data]

Creo il modello UNET

Classe DoubleConv

In [8]:
class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(DoubleConv, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    )

  def forward(self, x):
    return self.conv(x)



Classe UNET

In [9]:
class UNET(nn.Module):
  def __init__(
      self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
  ):
    super(UNET, self).__init__()
    self.ups = nn.ModuleList()
    self.downs = nn.ModuleList()
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    for feature in features:
      self.downs.append(DoubleConv(in_channels, feature))
      in_channels = feature

    for feature in reversed(features):
      self.ups.append(
          nn.ConvTranspose2d(
              feature*2, feature, kernel_size=2, stride=2,
          )
      )
      self.ups.append(DoubleConv(feature*2, feature))

    self.bottleneck = DoubleConv(features[-1], features[-1]*2)
    self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)
  def forward(self, x):
    skip_connections = []

    for down in self.downs:
      x = down(x)
      skip_connections.append(x)
      x = self.pool(x)

    x = self.bottleneck(x)
    skip_connections = skip_connections[::-1]

    for idx in range(0, len(self.ups), 2):
      x = self.ups[idx](x)
      skip_connection = skip_connections[idx//2]

      if x.shape != skip_connection.shape:
        x = TF.resize(x, size=skip_connection.shape[2:])

      concat_skip = torch.cat((skip_connection, x), dim=1)
      x = self.ups[idx+1](concat_skip)

    return self.final_conv(x)

Test

In [10]:
def test():
  x = torch.randn((3, 1, 160, 160))
  model = UNET(in_channels=1, out_channels=1)
  preds = model(x)
  print(preds.shape)
  print(x.shape)
  assert preds.shape == x.shape

if __name__ == "__main__":
  test()

torch.Size([3, 1, 160, 160])
torch.Size([3, 1, 160, 160])


Seed

In [10]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

Funzione per il training

In [16]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=device)
        targets = targets.float().to(device=device)

        with torch.cuda.amp.autocast():
            predictions = model(data).squeeze(1)
            loss = loss_fn(predictions, targets)


        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loop.set_postfix(loss=loss.item())

# def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=30, device='cuda', accumulation_steps=4):
#     best_f1 = 0.0

#     scaler = torch.cuda.amp.GradScaler()

#     for epoch in range(num_epochs):
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train()
#                 data_loader = train_loader
#             else:
#                 model.eval()
#                 data_loader = val_loader

#             running_loss = 0.0
#             all_preds = []
#             all_labels = []

#             optimizer.zero_grad()

#             for i, (inputs, labels) in enumerate(tqdm(data_loader, desc=f'{phase.capitalize()} Epoch {epoch+1}/{num_epochs}', leave=False)):
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

#                 with torch.cuda.amp.autocast():
#                     outputs = model(inputs).squeeze(1)
#                     loss = criterion(outputs, labels) / accumulation_steps
#                     preds = torch.sigmoid(outputs) > 0.5

#                 if phase == 'train':
#                     scaler.scale(loss).backward()

#                     if (i + 1) % accumulation_steps == 0:
#                         scaler.step(optimizer)
#                         scaler.update()
#                         optimizer.zero_grad()

#                 running_loss += loss.item() * inputs.size(0) * accumulation_steps
#                 all_preds.extend(preds.detach().cpu().numpy().flatten())
#                 all_labels.extend(labels.detach().cpu().numpy().flatten())


#             if phase == 'train' and (i + 1) % accumulation_steps != 0:
#                 scaler.step(optimizer)
#                 scaler.update()
#                 optimizer.zero_grad()

#             epoch_loss = running_loss / len(data_loader.dataset)

#             if phase == 'train':
#                 scheduler.step()
#             else:
#                 precision = precision_score(all_labels, all_preds)
#                 recall = recall_score(all_labels, all_preds)
#                 f1 = f1_score(all_labels, all_preds)
#                 accuracy = accuracy_score(all_labels, all_preds)

#                 if f1 > best_f1:
#                     best_f1 = f1
#                     torch.save(model.state_dict(), "/content/drive/MyDrive/best_model.pth")

#                 print(f"Epoch {epoch+1}/{num_epochs} - {phase.capitalize()} Loss: {epoch_loss:.4f} Precision: {precision:.4f} Recall: {recall:.4f} F1: {f1:.4f} Accuracy: {accuracy:.4f}")


Utils

In [12]:
def save_checkpoint(state, filename="/content/drive/MyDrive/checkpoint.pth.tar"):
  print("=> Saving checkpoint")
  torch.save(state, filename)

def load_checkpoint(checkpoint, model):
  print("=> Loading checkpoint")
  model.load_state_dict(checkpoint["state_dict"])

Accuracy

In [13]:
def check_accuracy(loader, model, loss_fn, device="cuda"):
  model.eval()
  num_correct = 0
  num_pixels = 0
  dice_score = 0
  precision = 0
  recall = 0
  f1 = 0
  val_loss = 0

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device)
      y = y.to(device).unsqueeze(1)
      preds = torch.sigmoid(model(x))
      preds = (preds > 0.5).float()
      num_correct += (preds == y).sum()
      num_pixels += torch.numel(preds)
      dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)
      precision += precision_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten())
      recall += recall_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten())
      f1 += f1_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten())
      val_loss += loss_fn(preds, y).item()

  val_loss /= len(loader)
  print(f"Validation Loss: {val_loss:.4f}")
  print(f"Accuracy: {num_correct/num_pixels*100:.2f}")
  print(f"Dice score: {dice_score/len(loader)}")
  print(f"Precision: {precision/len(loader)}")
  print(f"Recall: {recall/len(loader)}")
  print(f"F1 Score: {f1/len(loader)}")

  model.train()

In [14]:
def save_predictions_as_imgs(loader, model, folder, device="cuda"):
    os.makedirs(folder, exist_ok=True)
    model.eval()
    for idx, (x, y) in enumerate(tqdm(loader, desc="Saving predictions")):
        x = x.to(device=device)

        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()

        for i in range(preds.shape[0]):
            pred_tiff_path = os.path.join(folder, f"pred_{idx * loader.batch_size + i}.png")
            tiff.imwrite(pred_tiff_path, preds[i].cpu().numpy().astype('float32'))


        for i in range(y.shape[0]):
            label_tiff_path = os.path.join(folder, f"label_{idx * loader.batch_size + i}.png")
            tiff.imwrite(label_tiff_path, y[i].cpu().numpy().astype('float32'))

    model.train()


In [14]:
set_seed(42)

transform = A.Compose([
    A.Resize(height=128, width=128),
    # A.HorizontalFlip(p=0.5),
    # A.VerticalFlip(p=0.5),
    # A.Rotate(limit=90, p=0.5),
    A.Normalize(mean=mean, std=std),
    ToTensorV2(),
])

train_dataset = SARDataset(train_image_paths, train_label_paths, transform=transform)
val_dataset = SARDataset(val_image_paths, val_label_paths, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)
load_model = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNET(in_channels=4, out_channels=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = StepLR(optimizer, step_size=9)
num_epochs = 5
scaler = torch.cuda.amp.GradScaler()

if load_model:
  load_checkpoint(torch.load("/content/drive/MyDrive/checkpoint.pth.tar"), model)



In [17]:
for epoch in range(num_epochs):
        print(f"epoch {epoch}")
        train_fn(train_loader, model, optimizer, criterion, scaler)

        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

        check_accuracy(val_loader, model, criterion, device=device)

        scheduler.step()

epoch 0


  0%|          | 0/149 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 149/149 [12:05<00:00,  4.87s/it, loss=0.307]


=> Saving checkpoint
Validation Loss: 0.6935
Accuracy: 94.27
Dice score: 0.012126022949814796
Precision: 0.26289846031299946
Recall: 0.006270886591758128
F1 Score: 0.012126022426254941
epoch 1


 11%|█▏        | 17/149 [05:01<38:59, 17.72s/it, loss=0.325]


KeyboardInterrupt: 